# RMSE calculation - 6 hourly IFS

* This notebook shows how RMSEs were computed for 6 hourly IFS outputs.

* The notebook runs with `verif_config_6h.yml` in the same folder.

In [1]:
import os
import sys
import yaml
import argparse
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

### Load config

In [2]:
config_name = os.path.realpath('verif_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### Model and lead time to verify

In [3]:
# ====================== #
model_name = 'IFS'
lead_range = conf[model_name]['lead_range']
verif_lead_range = conf[model_name]['verif_lead_range']

leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
leads_verif = list(np.arange(verif_lead_range[0], verif_lead_range[-1]+verif_lead_range[0], verif_lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, leads_verif)

print('Verifying lead times: {}'.format(leads_verif))
print('Verifying lead indices: {}'.format(ind_lead))
# ====================== #

Verifying lead times: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
Verifying lead indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [4]:
verif_ind_start = 0; verif_ind_end = 2*(365+366+365)

path_verif = conf[model_name]['save_loc_verif']+'combined_rmse_{:04d}_{:04d}_{:03d}h_{:03d}h_{}.nc'.format(
    verif_ind_start, verif_ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)

### RMSE compute

In [6]:
# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')
    
# Select the specified variables and their levels
variables_levels = conf['ERA5_ours']['verif_variables']

# subset merged ERA5 and unify coord names
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]

L_max = len(filename_OURS)
print('Total number of initialization times: {}'.format(L_max))
assert verif_ind_end <= L_max, 'verified indices (days) exceeds the max index available'

filename_OURS = filename_OURS[verif_ind_start:verif_ind_end]

# latitude weighting
lat = xr.open_dataset(filename_OURS[0])["lat"]
w_lat = np.cos(np.deg2rad(lat))
w_lat = w_lat / w_lat.mean()
w_lat = w_lat.rename({'lat': 'latitude'})

Total number of initialization times: 2922


In [7]:
# ---------------------------------------------------------------------------------------- #
# RMSE compute
verif_results = []

for fn_ours in filename_OURS:
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variables_levels)
    ds_ours = ds_ours.isel(time=ind_lead)
    ds_ours = ds_ours.compute()

    # rename coordinate name to latitude, longitude
    if 'lat' in ds_ours.coords:
        ds_ours = ds_ours.rename({'lat': 'latitude'})
    if 'lon' in ds_ours.coords:
        ds_ours = ds_ours.rename({'lon': 'longitude'})
    
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time']).compute()
    
    # RMSE with latitude-based cosine weighting (check w_lat)
    RMSE = np.sqrt(
        (w_lat * (ds_ours - ds_target)**2).mean(['latitude', 'longitude'])
    )
    
    verif_results.append(RMSE.drop_vars('time'))
    
# Combine verif results
ds_verif = xr.concat(verif_results, dim='days')

# # Save the combined dataset
# print('Save to {}'.format(path_verif))
# ds_verif.to_netcdf(path_verif)

In [8]:
ds_verif['Z500']

<xarray.DataArray 'Z500' (days: 3, time: 40)>
array([[ 31.13889381,  40.85765675,  41.24889393,  54.76000996,
         58.95706652,  72.68258807,  78.4631235 ,  95.45350442,
        106.82370879, 121.04959438, 131.14634106, 152.15579198,
        172.75632625, 196.41817103, 216.50339155, 238.61309905,
        254.99959073, 274.79519413, 297.07775549, 325.98977089,
        348.68196348, 371.79364285, 391.6813483 , 412.8846439 ,
        434.5261248 , 461.80599675, 490.66436378, 519.69266456,
        550.8426672 , 580.09690813, 606.61573246, 632.57819623,
        663.46599561, 700.00409137, 741.69857608, 784.15402541,
        822.54560828, 857.93999328, 885.33488864, 910.9078555 ],
       [ 28.4583993 ,  40.16587196,  42.65506181,  57.22482527,
         61.37425178,  77.42601098,  84.54018958,  98.01907228,
        108.26066777, 125.42297855, 136.63590461, 153.70962663,
        166.8848883 , 187.18485686, 203.79202568, 224.95558347,
        247.24278306, 280.07120817, 312.02819242, 345.49029871,
        372.64960446, 399.89927584, 425.84329205, 447.1148713 ,
        465.81145488, 488.03053247, 516.12740054, 547.94559426,
        579.26541182, 614.68976224, 644.40598667, 680.14692238,
        717.81704897, 756.38213783, 785.98126312, 807.2141669 ,
        824.04089056, 844.88750032, 867.78825237, 894.97405038],
       [ 27.75103352,  39.82234442,  39.54720139,  54.52852138,
         57.76406041,  70.53357597,  77.0163992 ,  92.08214419,
        101.23092764, 116.78680716, 125.98535292, 146.45820883,
        160.06205036, 181.68429538, 198.21536204, 225.03541571,
        254.24146883, 289.65081425, 319.1498881 , 350.55686634,
        381.6631767 , 411.80526985, 444.91556371, 482.18580479,
        513.85760347, 542.58566137, 566.87230068, 588.7041228 ,
        605.84499918, 628.8262365 , 655.6923727 , 687.45587068,
        712.84714826, 739.96333335, 764.74828899, 794.11408063,
        824.37272763, 858.53092983, 883.59717051, 897.55005196]])
Dimensions without coordinates: days, time